# Lyric Generation

The code in this notebook generates the lyrics using the model from the previous notebooks. The code works by receiving a user input and then generating a continuation of the user's prompt.

## Load the model

Import the necessary libraries.

In [1]:
import torch
import numpy as np
from torch import nn
from transformers import GPT2Tokenizer, GPT2Config, GPT2Model, GPT2PreTrainedModel
from torch.optim import AdamW
from datasets import load_dataset
from tqdm import tqdm
from torch.nn import functional as F
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_built() else 'cpu'

/Users/selbl/anaconda3/envs/CS129/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the model and define the generation function.

In [2]:
class GPT2_Model(GPT2PreTrainedModel):

    def __init__(self, config):

        super().__init__(config)

        self.transformer = GPT2Model.from_pretrained('gpt2')
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|pad|>')

        # this is necessary since we add a new unique token for pad_token
        self.transformer.resize_token_embeddings(len(tokenizer))

        self.lm_head = nn.Linear(config.n_embd, len(tokenizer), bias=False)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):

        x = self.transformer(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0]
        x = self.lm_head(x)

        return x

In [3]:
#Load model
configuration = GPT2Config()
gpt_model = GPT2_Model(configuration).to(device)
gpt_model.load_state_dict(torch.load('GPT-Trained-Model'))
gpt_model.eval()
#Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|pad|>') 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Define the generation function. The model samples probable continuations to the user's input. You can play with the top_k and top_p parameters to change the outputs of the model. Lowering p and increasing k makes it so the model outputs more "out there" word combinations.

In [4]:
#Dedfine generation function
def generate(idx, max_new_tokens, context_size, tokenizer, model, top_k=10, top_p=0.95):

        for _ in range(max_new_tokens):
            if idx[:,-1].item() != tokenizer.encode(tokenizer.eos_token)[0]:
                # crop idx to the last block_size tokens
                idx_cond = idx[:, -context_size:]
                # get the predictions
                logits = model(idx_cond)
                # focus only on the last time step
                logits = logits[:, -1, :]
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1)
                # sort probabilities in descending order
                sorted_probs, indices = torch.sort(probs, descending=True)
                # compute cumsum of probabilities
                probs_cumsum = torch.cumsum(sorted_probs, dim=1)
                # choose only top_p tokens
                sorted_probs, indices = sorted_probs[:, :probs_cumsum[[probs_cumsum < top_p]].size()[0] + 1], indices[:, :probs_cumsum[[probs_cumsum < top_p]].size()[0] +1]
                # choose only top_k tokens
                sorted_probs, indices = sorted_probs[:,:top_k], indices[:,:top_k]
                # sample from the distribution
                sorted_probs = F.softmax(sorted_probs, dim=-1)
                idx_next = indices[:, torch.multinomial(sorted_probs, num_samples=1)].squeeze(0)
                # append new token ids
                idx = torch.cat((idx, idx_next), dim=1)
            else:
                break

        return idx

## Lyric Formatting

Having done all of the above, the code can now generate lyrics. Before doing so, and in order to keep things PG, I add a profanity filter (which you can disable at your own risk by changing the profanity boolean variable).

In [5]:
from better_profanity import profanity
profanity.load_censor_words()
prof = False

By default, the code adds a _[verse]_ tag to precede the user's prompt as well as block profanity. You can disable that prompting by changing the parts variable below. Keep in mind that the code formats the lyrics to make them more readable only if they include these parts (see the example below).

If you would like, you can also change the song so it does not start with a _[verse]_, but rather with _[chorus]_ or something more creative. You can change the prefix variable below for that.

In [6]:
#Set model parameters
parts = True
prefix = '[verse]'

In [7]:
#Input prompt
prompt = "I stare at the sun"
prompt = prompt.lower()

In [8]:
#Pre-process prompt
if parts:
    prompt = prefix + ' ' + prompt

If you care about formatting, you can output your generated songs using the following function. Note that this only works with songs that have a song structure.

In [9]:
import re

#Define capitalization functions
def custom_capitalize(match):
    return match.group(1).capitalize()

def capitalize_string(input_string):
    # Capitalize every first letter after "\n\n "
    result = re.sub(r'\n\n\s*([a-zA-Z])', lambda x: '\n\n' + x.group(1).upper(), input_string)
    # Capitalize every first letter in every word inside brackets ([ ])
    result = re.sub(r'\[([^\]]*)\]', lambda x: '[' + ' '.join(word.capitalize() for word in x.group(1).split()) + ']', result)
    # Capitalize every instance of the letter i by itself
    result = re.sub(r'\bi\b', 'I', result)
    return result

#Format the string
def format_string(input_string):
    inside_brackets = False
    result = ''.join([char + ('\n\n' if char == ']' else '') for char in input_string.replace('\n', '')])
    result = result.replace('[', '\n\n[')
    #Capitalize for good measure
    result = capitalize_string(result)
    return result

## Lyric Generation

After all the preliminaries, it is time to generate some lyrics! The following block generates the lyrics. Play around with different prompts and try tweaking all of the parameters!

In [10]:
gpt_model.eval()
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

sample_outputs = generate(generated,
                         max_new_tokens=200,
                         context_size=400,
                         tokenizer=tokenizer,
                         model=gpt_model,
                         top_k=10,
                         top_p=0.95)

In [11]:
#Store the lyrics and decode
lyric = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
#Format if it has parts
if parts:
    lyric = format_string(lyric)
#Remove profanity
if not prof:
    lyric = profanity.censor(lyric)
print(lyric)



[Verse]

I stare at the sun every night up ahead, oh, look at the clouds all these clouds under the hood  

[Pre-chorus]

They cover my face with bad cheap leather and then shake my hat and wash my arm then shake my hands  

[Chorus]

Now I find myself in the back of a silver highway and the skies are green and the nights are blue and I'm walking up that highway I body go jumping, he calls, he calls a laughing back  

[Verse]

Now my shoes are gray and my head goes brown but I'm not joking about the awful talking in the back yard, hey now he did it to you the gocks all my hands are dragging, they whisper, "we'll save you who you want to be you're being alright with pretending not stealing how you like that  

[Pre-chorus]

But the crowd can't forget you the big seats are falling down and the kids are putting it down  

[Chorus]

Now I find myself in the back of


I hope this model works for you! There's a lot to explore by generating lyrics with this method. While this generator is far from perfect, it can give you a great starting point for your next composition. If you come up with anything, please send it my way to give it a spin!